# Corridor experiments

In this notebook, we look at matchings with corridor navground robots. 

First, let us import a few important modules for this task.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist
import matplotlib as mpl

import os

from navground import core, sim

import perdiver.perdiver as perdiver
from perdiver.distances import *

plots_dir = "plots/corridor/"
os.makedirs(plots_dir, exist_ok=True)

Next, let us execute the Navground corridor experiment.

In [ ]:
length = 8.0
num_steps = 1000
# num_steps = 0
width=1.0
yaml = f"""
steps: {num_steps}
time_step: 0.1
save_directory: ''
record_pose: true
record_twist: true
scenario:
  type: Corridor
  length: {length}
  width: {width} 
  groups:
    -
      type: thymio
      number: 38
      radius: 0.08
      control_period: 0.1
      speed_tolerance: 0.02
      kinematics:
        type: 2WDiff
        wheel_axis: 0.094
        max_speed: 0.166
      behavior:
        type: HL
        optimal_speed: 0.12
        horizon: 5.0
        safety_margin: 0.034
      state_estimation:
        type: Bounded
        range: 5.0
"""
experiment = sim.load_experiment(yaml)
experiment.run()

We read out the positions and velocities of robots during the simulation.

In [ ]:
run = experiment.runs[0]
ps = run.poses[:,:,[0,1]]
twists = run.twists[:,:,:2] # ignore angular speeds

Let us see a short simulation video.

In [ ]:
# import matplotlib.colors as colors
# import matplotlib.cm as cmx
# from navground.sim.ui.video import record_video

# def linear_map(a, b, cmap):
#     c = cmx.ScalarMappable(norm=colors.Normalize(vmin=a, vmax=b), cmap=cmap)  
#     def f(v):
#         r, g, b, _ = c.to_rgba(v)
#         return f"#{int(r * 255):02x}{int(g * 255):02x}{int(b * 255):02x}"
#     return f


# fill_map = linear_map(0.0, 1.0, cmap=cmx.RdYlGn)

# def f(entity):
#     if isinstance(entity, sim.Agent):
#         return {'fill': fill_map(entity.behavior.efficacy)}
#     return {}
    
# record_video("stuck_corridor_exp.mp4", run.world, time_step=0.1, duration=60.0, factor=8.0,
#               bounds=((0, 0), (length, width)), decorate=f, width=700, display_shape=True)

We observe mainly two dynamics, either the robots end up going in straight trajectories or they get stuck. Both examples below where produced with the same variables.

In [ ]:
# from IPython.display import Video

# Video("stuck_corridor_exp.mp4", width=700)

In [ ]:
# from IPython.display import Video

# Video("straight_corridor_exp.mp4", width=700)

### Pairwise Matchings

We are now going to compute the induced matchings and their associated diagrams.

Also, we set up the variable "weight" and the timestep shift for our experiments.

In [ ]:
weight = 4
shift_time = 30
steps_list = list(range(0, 800, 5))

Now, we are going to start by considering two timesteps and their associated divergence diagrams. Notice that these do not change much.

In [ ]:
start_step = 900
X = ps[start_step]
Y = ps[start_step + shift_time]
vel_X = twists[start_step]
vel_Y = twists[start_step + shift_time]
X_len = X.shape[0]-1
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 4))
# Interpret diagram via points
perdiver.plot_two_timesteps(X, Y, ax[0])
Dist_X, Dist_Y, Dist_Z = perdiver.compute_distance_matrices_timesteps_corridor(X, Y, vel_X, vel_Y, weight, length)
match_diagram = perdiver.get_matching_diagram(Dist_X, Dist_Y)
perdiver.plot_matching_diagram(match_diagram, ax[1], color="blue")

Next, we compute and print the divergence array across a few simulation steps.

In [ ]:
divergence_list = []
Z_barcodes_list = []
for start_step in steps_list:
    X = ps[start_step]
    Y = ps[start_step + shift_time]
    vel_X = twists[start_step]
    vel_Y = twists[start_step + shift_time]
    Dist_X, Dist_Y, Dist_Z = perdiver.compute_distance_matrices_timesteps_corridor(X, Y, vel_X, vel_Y, weight, length)
    divergence_vector = perdiver.compute_divergence_vector(Dist_X, Dist_Y)
    divergence_list.append(divergence_vector)

divergence_arr = np.array(divergence_list).transpose()

In [ ]:
fig, ax = plt.subplots(figsize=(10,2))
vmax = np.max(divergence_arr)
mapable = ax.imshow(divergence_arr, aspect="auto", vmax=vmax, extent=(steps_list[0], steps_list[-1], 0, X.shape[0]))
ax.set_title("Divergence")
plt.colorbar(mapable)
plt.tight_layout()
ax.set_title("Divergence across time")
plt.savefig("Divergence_vector_evolution.png")
# ax.set_title("Divergence constant speed example")
# plt.savefig("Divergence_corridor_constant.png")

Last, we compute the persistence matching diagram across the step list. There seems to be an equilibrium.

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
for idx, start_step in enumerate(steps_list):
    X = ps[start_step]
    Y = ps[start_step + shift_time]
    vel_X = twists[start_step]
    vel_Y = twists[start_step+shift_time]
    Dist_X, Dist_Y, Dist_Z = perdiver.compute_distance_matrices_timesteps_corridor(X, Y, vel_X, vel_Y, weight, length)
    match_diagram = perdiver.get_matching_diagram(Dist_X, Dist_Y)
    perdiver.plot_matching_diagram(match_diagram, ax, color=mpl.colormaps["GnBu"](idx/len(steps_list)))
# plt.savefig("pers_div_diag_constant.png")
norm = mpl.colors.Normalize(vmin=steps_list[0], vmax=steps_list[-1])
cmap = mpl.colormaps["GnBu"]
mappable = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
plt.colorbar(mappable=mappable, ax=ax)
# plt.savefig("pers_div_diag_constant.png")
# plt.savefig("pers_div_diag_stuck.png")

### Trajectories analysis

Now, we repeat the same experiment, but, instead of considering timesteps, we consider consecutive trajectories.

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
start_step=300
Dist_X, Dist_Y, Dist_Z = compute_distance_matrices_trajectories_corridor(ps, twists, start_step, shift_time, weight, length)
perdiver.plot_matching_diagram(Dist_X, Dist_Y, Dist_Z, ax, color="blue", print_barcode_n_reps=True)
plt.savefig(plots_dir + "divergence_diag.png")

In [ ]:
X_seq = ps[list(range(start_step, start_step+shift_time+1, 10))]
fig, ax = plt.subplots(figsize=(10, 5))
ax.set_aspect("equal")
perdiver.plot_sequence(X_seq, ax)

In [ ]:
divergence_list = []
Z_barcodes_list = []
for start_step in steps_list:
    Dist_X, Dist_Y, Dist_Z = compute_distance_matrices_trajectories_corridor(ps, twists, start_step, shift_time, weight, length)
    divergence, Z_barcode = perdiver.compute_divergence_vector(Dist_X, Dist_Y, Dist_Z)
    divergence_list.append(divergence)
    Z_barcodes_list.append(Z_barcode)

divergence_arr = np.array(divergence_list).transpose()
Z_barcodes_arr = np.array(Z_barcodes_list).transpose()

Next, we plot divergence vectors over time.

In [ ]:
fig, ax = plt.subplots(figsize=(10,2))
vmax = np.max(divergence_arr)
mapable = ax.imshow(divergence_arr, aspect="auto", vmax=vmax, extent=(steps_list[0], steps_list[-1], 0, divergence_arr.shape[0]))
print(mapable)
print(np.max(divergence_arr))
ax.set_title("corridor")
plt.colorbar(mapable)
plt.tight_layout()
plt.savefig("corridor_divergence_diag_hist.png")

In [ ]:
fig, ax = plt.subplots(figsize=(10,2))
vmax = np.max(Z_barcodes_arr)
mapable = ax.imshow(Z_barcodes_arr, aspect="auto", vmax=vmax, extent=(steps_list[0], steps_list[-1], 0, Z_barcodes_arr.shape[0]))
ax.set_title("corridor")
plt.colorbar(mapable)
plt.tight_layout()
plt.savefig("corridor_Z_array_hist.png")

Let us plot the persistence diagram over time for trajectories.

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
for idx, start_step in enumerate(steps_list):
    Dist_X, Dist_Y, Dist_Z = compute_distance_matrices_trajectories_corridor(ps, twists, start_step, shift_time, weight, length)
    perdiver.plot_matching_diagram(Dist_X, Dist_Y, Dist_Z, ax, color=mpl.colormaps["GnBu"](idx/len(steps_list)))


In [ ]:
# Persistence Images for corridor experiment

In [ ]:
from importlib import reload
reload(perdiver)

In [ ]:
diags = []
for idx, start_step in enumerate(steps_list):
    Dist_X, Dist_Y, Dist_Z = compute_distance_matrices_trajectories_corridor(ps, twists, start_step, shift_time, weight, length)
    diags.append(perdiver.get_matching_diagram(Dist_X, Dist_Y, Dist_Z, ax, color=mpl.colormaps["GnBu"](idx/len(steps_list))))

# Persistence Images

In [ ]:
length = 8.0
num_steps = 2000
# num_steps = 0
width=1.0
yaml = f"""
steps: {num_steps}
time_step: 0.1
save_directory: ''
record_pose: true
record_twist: true
scenario:
  type: Corridor
  length: {length}
  width: {width} 
  groups:
    -
      type: thymio
      number: 38
      radius: 0.08
      control_period: 0.1
      speed_tolerance: 0.02
      kinematics:
        type: 2WDiff
        wheel_axis: 0.094
        max_speed: 0.166
      behavior:
        type: HL
        optimal_speed: 0.12
        horizon: 5.0
        safety_margin: 0.034
      state_estimation:
        type: Bounded
        range: 5.0
"""
experiment = sim.load_experiment(yaml)
experiment.run()
run = experiment.runs[0]
ps = run.poses[:,:,[0,1]]
twists = run.twists[:,:,:2] # ignore angular speeds

In [ ]:
diags = []
steps_list = list(range(0, 1400, shift_step))
for idx, start_step in enumerate(steps_list):
    Dist_X, Dist_Y, Dist_Z = compute_distance_matrices_trajectories_corridor(ps, twists, start_step, shift_time, weight, length)
    diags.append(perdiver.get_matching_diagram(Dist_X, Dist_Y, Dist_Z, ax, color=mpl.colormaps["GnBu"](idx/len(steps_list))))

In [ ]:
from gudhi import representations

In [ ]:
shift_step = 10
diag_sum_list = []
list_step = int(100/shift_step)
for start_idx in range(0,len(steps_list), list_step ):
    diag_sum_list.append(np.vstack(diags[start_idx: start_idx+list_step ]))

diag_sum_list = diag_sum_list[:10]

In [ ]:
npixels = 30
perim = representations.PersistenceImage(resolution=[npixels, npixels], bandwidth=0.06)
perim.fit(diag_sum_list)
perim_list = perim.transform(diag_sum_list)
# print(diag_trans.shape)
# diag_shift_list = []
# for diag in diags:
#     diag_trans = np.vstack((diag[:,0], diag[:,1])).transpose()
#     diag_shift_list.append(diag_trans)
    
# perim.fit(diag_shift_list)
# perim_list = perim.transform(diag_shift_list)
for image, diag in zip(perim_list, diag_sum_list):
    fig, ax = plt.subplots(ncols=2, figsize=(4,2))
    image = image.reshape(npixels,-1)
    ax[0].imshow(image)
    ax[1].scatter(diag[:,0], diag[:,1])
    ax[1].set_aspect("equal")